# iPython Nutrition Notebook

This notebook was created to make it easier to understand how several behavioral variables affect your body's composition. Please add or remove from it what you like, as it was only ever intended to aid the otherwise obnoxiously tedious activity that is macro tracking.

## Initial setup
Start MongoDB and run this cell before interacting with the rest of this notebook.

In [1]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt

#Start MongoDB Client
client = MongoClient()
db = client.primer
coll = db.dataset

#FOODS

#Simple Foods
empty = np.array([0.,0.,0.,0.])
Egg = np.array([78., 5., 0.6, 6.])
Avocado_whole = np.array([234., 21., 12., 2.9])
Sour_Cream_1tbsp = np.array([23., 2.4, 0.3, 0.2])
Tillamook_Butter_1Gr = np.array([100/14., 12/14., 0, 0])
EarthBalance_Butter_1Gr = np.array([100/14., 11/14., 0, 0])
Spinach = np.array([7., 0.1, 1.1, 0.9])
Chicken_1oz = np.array([32.5, 0.75, 0, 6.5])
Tofu_1oz = np.array([40., 2., 1., 4.666])
Peanut_Butter_2tbsp = np.array([200., 16., 6., 8.])
Refried_Beans_1serving = np.array([195, 0.89, 33.7, 10.64])
Trail_Mix_hfcup = np.array([360, 24., 31.2, 12.])
Trail_Mix_30Gr = np.array([150., 10., 13. , 5.])
Protein_Powder_1scoop = np.array([140., 2., 6., 24.])
Oatmeal_hfcup = np.array([150.0, 3.0, 27.0, 5.0])
Sweet_Potato_1oz = np.array([86./3.5274, 0.1/3.5274, 5.6698985088, 1.6/3.5274])
Potato_1oz = np.array([100/3.5274, 0.1/3.5274, 17/3.5274, 2/3.5274])
White_Rice_1cp = np.array([206, 0.4, 45.0, 4.3])
One_Percent_Milk_1cup = np.array([130, 2.5, 16., 11.])
Two_Percent_Milk_1cup = np.array([130., 5., 13., 10.])
Plain_Soymilk_1cup = np.array([100., 4., 8., 7.])
AltaDenaOnePercentMilk = np.array([90*8, 0, 13*8, 9*8])
Cottage_Cheese_hfcup = np.array([80.0, 0.0, 6.0, 13.0])
Brown_Rice_Flour_1cup = np.array([140.*4, 4., 31.*4., 3.*4.])
Cornstarch_1tbsp = np.array([30., 0., 7., 0.])

#Protein
Amplified_Mass_1scoop = np.array([187.5, 1.5, 31, 12.5])
Protein_Powder_MochaCappuccino = np.array([120, 1.5, 3., 24.])
Protein_Powder_DoubleChocolate = np.array([120, 1., 3., 24.])
Protein_Powder_ExtraMilkChocolate = np.array([120, 1.5, 3, 24.])

#Mealy Foods
chicken_serving_small = 8*Chicken_1oz
chicken_serving_large = 10*Chicken_1oz
halfAnAvocado = np.multiply(0.5, Avocado_whole)
quarterAnAvocado = np.multiply(0.25, Avocado_whole)
omelette = 6*Egg + halfAnAvocado + 6*Sour_Cream_1tbsp + Spinach
omeletteLessFat = 6*Egg + quarterAnAvocado + Spinach + Refried_Beans_1serving
vegieOmeletteLessFat = 6*Tofu_1oz + Plain_Soymilk_1cup/2. + 0.0625011*Brown_Rice_Flour_1cup + Cornstarch_1tbsp + halfAnAvocado + Spinach + Refried_Beans_1serving
oatmeal = 2*Oatmeal_hfcup
halfMass = 2*Amplified_Mass_1scoop
halfMassWithMilk = halfMass + One_Percent_Milk_1cup

###### Enums ######
#Macronutrients
Fat, Carbohydrates, Protein = range(3)
#Gender
Male, Female = range(2)
#Activity Level
Sedentary, Light, Moderate, VeryActive = range(4)
#Goals
WeightLoss, Maintenance, Bulking = [0.8, 1.0, 1.1]

    
#utility functions
def lbsToKG(weightInLbs):
    return weightInLbs*0.453592

def inToCm(heightInIn):
    return heightInIn*2.54

def calculateDailyCalories(weightInLbs, heightInIn, age, gender, activityLevel, goal):
    restingEnergyExpenditure = 0.
    if gender == Male:
        restingEnergyExpenditure = 10*lbsToKG(weightInLbs) + 6.25*inToCm(heightInIn) - 5*age + 5.
    elif gender == Female:
        restingEnergyExpenditure = 10.*lbsToKG(weightInLbs) + 6.25*inToCm(heightInIn) - 5.*age - 161.
        
    totalDailyEnergyExpenditure = ([1.2, 1.375, 1.55, 1.725][activityLevel]) * restingEnergyExpenditure
    
    if total[0] > totalDailyEnergyExpenditure*goal:
        print "Warning. Over daily calorie budget by " + `total[0] - totalDailyEnergyExpenditure*goal` + " calories."
    
    return totalDailyEnergyExpenditure*goal

def calculateGramsOfMacro(targetCalories, currentMacros, macroString, weightInLbs, activityLevel):
    gramsProtein = 0.0
    gramsCarbs = 0.0
    gramsFat = 0.0
    
    if activityLevel <= 1:
        gramsProtein = weightInLbs*0.825
    else:
        gramsProtein = weightInLbs
    gramsFat = (targetCalories * 0.25)/9.0
    gramsCarbs = (targetCalories - (gramsFat*9.0 + gramsProtein*4.0))/4.0 
    
    returnArray = [gramsFat, gramsCarbs, gramsProtein]
    
    return returnArray[eval(macroString)]

def calculateGramsOfMacroLeft(targetCalories, currentMacros, macroString, weightInLbs, activityLevel):
    gramsProtein = calculateGramsOfMacro(targetCalories, currentMacros, "Protein", weightInLbs, activityLevel)
    gramsCarbs = calculateGramsOfMacro(targetCalories, currentMacros, "Carbohydrates", weightInLbs, activityLevel)
    gramsFat = calculateGramsOfMacro(targetCalories, currentMacros, "Fat", weightInLbs, activityLevel)
    
    gramsProtein = gramsProtein - currentMacros[3]
    gramsCarbs = gramsCarbs - currentMacros[2]  
    gramsFat = gramsFat - currentMacros[1]
    
    caloriesToSubtract = currentMacros[0] - targetCalories
    constantMultiplier = [9,4,4]
    returnArray = [gramsFat, gramsCarbs, gramsProtein]
    
    if caloriesToSubtract > 0:
        return 0
    else:
        return returnArray[eval(macroString)]

def calculateServingsOfFoodLeftByMacro(targetCalories, targetMacroAmountInGrams, currentMacros, macroString, food):
    gramsLeft = 0.0
    servings = 0.0
    
    gramsLeft = targetMacroAmountInGrams - currentMacros[eval(macroString)+1]
    servings = gramsLeft/eval(food)[eval(macroString)+1]
    
    if servings < 0:
        print "Over " + `macroString` + " budget by " + `-gramsLeft` + " grams."
        print "Equivalent to " + `-gramsLeft/eval(food)[eval(macroString)+1]` + " servings of " + `food`
    elif (targetCalories - currentMacros[0]) < 0:
        return 0
    
    return servings
    

## Tracking
This cell can be used for day-to-day tracking.

In [4]:
#Beginning-of-day tracking
weight = 160.2
bodyfatpercentage = 10
sleep = {"hours": 8, "minutes": 0}

#Meal tracking
firstMeal = Protein_Powder_MochaCappuccino + vegieOmeletteLessFat + 4*Plain_Soymilk_1cup
print("First Meal Calories: " + `firstMeal[0]`)
print("First Meal Fat: " + `firstMeal[1]`)
print("First Meal Carbs: " + `firstMeal[2]`)
print("First Meal Protein: " + `firstMeal[3]` + "\n")

secondMeal = 4*oatmeal
print("Second Meal Calories: " + `secondMeal[0]`)
print("Second Meal Fat: " + `secondMeal[1]`)
print("Second Meal Carbs: " + `secondMeal[2]`)
print("Second Meal Protein: " + `secondMeal[3]` + "\n")

thirdMeal = empty
print("Third Meal Calories: " + `thirdMeal[0]`)
print("Third Meal Fat: " + `thirdMeal[1]`)
print("Third Meal Carbs: " + `thirdMeal[2]`)
print("Third Meal Protein: " + `thirdMeal[3]` + "\n")

fourthMeal = empty
print("Fourth Meal Calories: " + `fourthMeal[0]`)
print("Fourth Meal Fat: " + `fourthMeal[1]`)
print("Fourth Meal Carbs: " + `fourthMeal[2]`)
print("Fourth Meal Protein: " + `fourthMeal[3]` + "\n")

fifthMeal = empty
print("Fourth Meal Calories: " + `fourthMeal[0]`)
print("Fourth Meal Fat: " + `fourthMeal[1]`)
print("Fourth Meal Carbs: " + `fourthMeal[2]`)
print("Fourth Meal Protein: " + `fourthMeal[3]` + "\n")

total = firstMeal + secondMeal + thirdMeal + fourthMeal + fifthMeal
print("Total Calories: " + `total[0]`)
print("Total Fat: " + `total[1]`)
print("Total Carbs: " + `total[2]`)
print("Total Protein: " + `total[3]`)

First Meal Calories: 1194.000616
First Meal Fat: 43.240004400000004
First Meal Carbs: 100.55013640000001
First Meal Protein: 97.236013200000002

Second Meal Calories: 1200.0
Second Meal Fat: 24.0
Second Meal Carbs: 216.0
Second Meal Protein: 40.0

Third Meal Calories: 0.0
Third Meal Fat: 0.0
Third Meal Carbs: 0.0
Third Meal Protein: 0.0

Fourth Meal Calories: 0.0
Fourth Meal Fat: 0.0
Fourth Meal Carbs: 0.0
Fourth Meal Protein: 0.0

Fourth Meal Calories: 0.0
Fourth Meal Fat: 0.0
Fourth Meal Carbs: 0.0
Fourth Meal Protein: 0.0

Total Calories: 2394.0006160000003
Total Fat: 67.240004400000004
Total Carbs: 316.55013640000004
Total Protein: 137.2360132


## Insertion
Use this cell to insert data into the database.

In [17]:
try:
  db, total
except NameError:
  print "Make sure to run the initial setup and tracking cells before this one."
else:
    daysSinceEpoch = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days

    result = db.macros.insert_one(
        {
            "date": daysSinceEpoch,
            "macros": {
                "calories": total[0],
                "fat": total[1],
                "carbohydrates": total[2],
                "protein": total[3]
            },
            "weight": weight,
            "bodyfat%": bodyfatpercentage,
            "sleep": sleep
        }
    )
    print "Insertion successful."

Insertion successful.


In [15]:
db.macros.delete_one({'date': 16904})

In [5]:
#db.macros.update_one({'date': 16873}, {'$set': {'weight': 162.2}})

## Data retrieval
Use this cell to view all data collected over a 'daysShown'-day period. The graping cell also depends on the output of this cell, so run this one first before graphing your data.

In [18]:
try:
  pd, db
except NameError:
  print "Make sure to run the initial setup cell before this one."
else:
    #lists to store tracked variables
    daysList = []
    caloriesList = []
    fatList = []
    carbsList = []
    proteinList = []
    sleepList = []
    weightList = []

    #setup for mongo search
    daysShown = 15
    earliestDate = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - daysShown
    cursor = db.macros.find( { 'date': { '$gt': earliestDate } } )

    #add each entry's fields to the respective lists
    for document in cursor:
        daysList.append(document['date'])
        caloriesList.append(document['macros']['calories'])
        fatList.append(document['macros']['fat'])
        carbsList.append(document['macros']['carbohydrates'])
        proteinList.append(document['macros']['protein'])
        weightList.append(document['weight'])
        sleepList.append((document['sleep']['hours'], document['sleep']['minutes']))
        print(document)

    combinedSleepList = []
    for (hours, minutes) in sleepList:
        combinedSleepList.append(hours+minutes/60.)
    

{u'weight': 160.0, u'macros': {u'carbohydrates': 396.02516300928005, u'calories': 3044.073915226917, u'protein': 173.47501304076658, u'fat': 87.96741152933363}, u'sleep': {u'hours': 6, u'minutes': 30}, u'bodyfat%': 10, u'date': 16890, u'_id': ObjectId('56fcc3a0a93ba305f4dc12ad')}
{u'weight': 158.6, u'macros': {u'carbohydrates': 473.3234538570667, u'calories': 3349.1068334487077, u'protein': 166.3168785966094, u'fat': 95.54458527300237}, u'sleep': {u'hours': 8, u'minutes': 10}, u'bodyfat%': 10, u'date': 16895, u'_id': ObjectId('57035ce9a93ba33a1c0e1f48')}
{u'weight': 158.6, u'macros': {u'carbohydrates': 473.55189406784007, u'calories': 3338.822459688998, u'protein': 162.200153813483, u'fat': 95.71072747405697}, u'sleep': {u'hours': 8, u'minutes': 10}, u'bodyfat%': 10, u'date': 16896, u'_id': ObjectId('5704b051a93ba33350fa40af')}
{u'weight': 158.6, u'macros': {u'carbohydrates': 472.84894005056, u'calories': 3332.2854717010828, u'protein': 156.46391749211315, u'fat': 97.69149841825708}, u

## Graphing.
This cell can be used to graph your data!

In [19]:
try:
  caloriesList, dt
except NameError:
  print "Make sure to run the initial setup and data retrieval cells before this one."
else:
    today = dt.date.today()

    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
    ax1.plot(daysList, caloriesList, color='y')
    ax2.plot(daysList, weightList, color='b')
    ax3.plot(daysList, carbsList, color='r')
    ax4.plot(daysList, combinedSleepList, color='g')

    ax1.set_ylim([0, max(caloriesList)*1.4])
    ax2.set_ylim([min(weightList), max(weightList)*1.1])
    ax3.set_ylim([0, max(carbsList)*1.4])
    ax4.set_ylim([min(combinedSleepList), max(combinedSleepList)*1.3])

    ax1.set_title('Calories')
    ax2.set_title('Weight')
    ax3.set_title('Carbohydrates')
    ax4.set_title('Sleep')


    plt.savefig(`today.month` + "-" + `today.day` + "-" + `today.year` + ".png")


## Useful Calculations
The following cell may be useful for doing initial and recurring macronutrient calculations.

In [5]:
try:
  Male, Female
except NameError:
  print "Make sure to run the initial setup cell before this one."
else:
    fatFoodName = "EarthBalance_Butter_1Gr"
    carbFoodName = "Sweet_Potato_1oz"
    proteinFoodName = "Protein_Powder_MochaCappuccino"
    goalWeight = 170
    heightInIn = 72
    gender = Male
    age = 22
    activityLevel = VeryActive
    goal = Bulking
    
try:
    total
except NameError:
    print "Make sure to run the tracking cell before running this one."
else:
    calories = calculateDailyCalories(goalWeight, heightInIn, age, gender, activityLevel, goal)
    gramsOfFatLeftToConsume = calculateGramsOfMacroLeft(calories, total, "Fat", goalWeight, activityLevel)
    gramsOfFatTotal = calculateGramsOfMacro(calories, total, "Fat", goalWeight, activityLevel)
    gramsOfCarbsLeftToConsume = calculateGramsOfMacroLeft(calories, total, "Carbohydrates", goalWeight, activityLevel)
    gramsOfCarbsTotal = calculateGramsOfMacro(calories, total, "Carbohydrates", goalWeight, activityLevel)
    gramsOfProteinLeftToConsume = calculateGramsOfMacroLeft(calories, total, "Protein", goalWeight, activityLevel)
    gramsOfProteinTotal = calculateGramsOfMacro(calories, total, "Protein", goalWeight, activityLevel)

    print "Daily calories: " + `calories`
    print "Current calories: " + `total[0]`
    print "Calories left: " + `calories-total[0]` + "\n"
    
    print "Grams of carbohydrates left: " + `gramsOfCarbsLeftToConsume`
    print "Additional servings of " + `carbFoodName` + " to meet daily carbohydrate intake: " + `calculateServingsOfFoodLeftByMacro(calories, gramsOfCarbsTotal, total, "Carbohydrates", carbFoodName)` + "\n"
    print "Grams of fat left: " + `gramsOfFatLeftToConsume`
    print "Additional servings of " + `fatFoodName` + " to meet daily fat intake: " + `calculateServingsOfFoodLeftByMacro(calories, gramsOfFatTotal, total, "Fat", fatFoodName)` + "\n"
    print "Grams of protein left: " + `gramsOfProteinLeftToConsume`
    print "Additional servings of " + `proteinFoodName` + " to meet daily protein intake: " + `calculateServingsOfFoodLeftByMacro(calories, gramsOfProteinTotal, total, "Protein", proteinFoodName)` + "\n"

Daily calories: 3432.779394000001
Current calories: 2394.0006160000003
Calories left: 1038.7787780000008

Grams of carbohydrates left: 157.09599997500015
Additional servings of 'Sweet_Potato_1oz' to meet daily carbohydrate intake: 27.707021515672345

Grams of fat left: 28.114978766666695
Additional servings of 'EarthBalance_Butter_1Gr' to meet daily fat intake: 35.782700248484886

Grams of protein left: 32.763986799999998
Additional servings of 'Protein_Powder_MochaCappuccino' to meet daily protein intake: 1.3651661166666667



In [23]:
print total[0]

2634.78338153
